In [20]:
import pandas as pd
import numpy as np
import plotly.express as px
from PIL import Image

import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.nn import functional as FractionalMaxPool3d
from torch import optim

from torch.utils.data import DataLoader,Dataset

from sklearn.model_selection import train_test_split


In [2]:
TRAIN_DATA = '../data/train.csv'
TEST_DATA = '../data/test.csv'
TRAIN_IMG = '../data/train_test_data/train/'

In [3]:
train_df = pd.read_csv(TRAIN_DATA)
train_df.head()

,label,latitude,longitude,year,example_path
0,0,-2.051853,111.826093,2001,train_test_data/train/1297.png
1,2,-1.989349,105.309496,2013,train_test_data/train/1199.png
2,0,1.223256,100.702217,2014,train_test_data/train/1348.png
3,0,-2.342948,103.890226,2008,train_test_data/train/2214.png
4,0,-0.126555,101.758175,2011,train_test_data/train/2220.png


In [4]:
test_df = pd.read_csv(TEST_DATA)
test_df.head()

,latitude,longitude,year,example_path
0,0.761681,122.755954,2006,train_test_data/test/69.png
1,-8.059785,113.053791,2007,train_test_data/test/469.png
2,-2.006610,111.746316,2002,train_test_data/test/6.png
3,0.901765,114.042495,2016,train_test_data/test/351.png
4,1.911210,100.829633,2008,train_test_data/test/1001.png


In [21]:
img = Image.open(TRAIN_IMG+'2220.png')
img_array = np.asarray(img)
px.imshow(img_array)

In [22]:
img_array[0][0]

array([35, 31, 14], dtype=uint8)

In [14]:
img.size

(332, 332)

In [6]:
img.mode

'RGB'

In [35]:
def imshow(img):
  img = img / 2 + 0.5 
  npimg = img.numpy() 
  px.imshow(np.transpose(npimg, (1, 2, 0)))
  

# Custom Dataset and Dataloader

In [34]:
class ImageDataset(Dataset):
  def __init__(self,df,data_folder,transform):
    self.df = df
    self.transform = transform
    self.img_folder = data_folder
     
    self.image_names = self.df[:]['example_path']
    self.labels = self.df[:]['label']
   
  def __len__(self):
    return len(self.image_names)
 
  def __getitem__(self,index):
     
    image=Image.open(self.img_folder+self.image_names.iloc[index])

    image=self.transform(image)
    targets=self.labels[index]
     
    sample = {'image': image,'labels':targets}
 
    return sample

train_transform = transforms.Compose([
                       transforms.ToTensor(), 
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_data = ImageDataset(train_df, '../data/', train_transform)
train_DL = DataLoader(train_data, batch_size=20,shuffle=True)
sample = next(iter(train_DL))
print(sample['labels'][0])
img = (sample['image'][0] / 2 + 0.5 )
npimg = img.numpy() 
px.imshow(np.transpose(npimg, (1, 2, 0)))

tensor(0)


AttributeError: module 'plotly.express' has no attribute 'show'

In [16]:
list(sample['image'][0].shape)

[3, 332, 332]

In [17]:
from model import Net

net = Net(list(sample['image'][0].shape), classes=3)
net

Net(
  (conv): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=102400, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=3, bias=True)
  )
)

In [19]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


# TRAINING

In [ ]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

start.record()

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_DL, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss_func = loss_func(outputs, labels)
        loss_func.backward()
        optimizer.step()

        # print statistics
        running_loss += loss_func.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

# whatever you are timing goes here
end.record()

# Waits for everything to finish running
torch.cuda.synchronize()

print('Finished Training')
print(start.elapsed_time(end))  # milliseconds